# Experimenting with Cohere notebook 
Taken from https://github.com/cohere-ai/notebooks/blob/main/notebooks/Text_Classification_Using_Embeddings.ipynb

In [19]:
!pip install cohere

## 1. Get the dataset

In [17]:
import pandas as pd
import cohere
from sklearn.model_selection import train_test_split
pd.set_option('display.max_colwidth', 500)

# Get the SST2 training and test sets
df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)

In [18]:
# Let's glance at the dataset
df.head()

,0,1
0,"a stirring , funny and finally transporting re imagining of beauty and the beast and 1930s horror films",1
1,apparently reassembled from the cutting room floor of any given daytime soap,0
2,"they presume their audience wo n't sit still for a sociology lesson , however entertainingly presented , so they trot out the conventional science fiction elements of bug eyed monsters and futuristic women in skimpy clothes",0
3,"this is a visually stunning rumination on love , memory , history and the war between art and commerce",1
4,jonathan parker 's bartleby should have been the be all end all of the modern office anomie films,1


In [20]:
num_examples = 500
df_sample = df.sample(num_examples)

# Split into training and testing sets
sentences_train, sentences_test, labels_train, labels_test = train_test_split(
            list(df_sample[0]), list(df_sample[1]), test_size=0.25, random_state=0)

## 2. Get the embeddings of the reviews

In [21]:
# I've hidden the key but you can get it from os.cohere.ai
api_key = ""
co = cohere.Client(api_key)

In [22]:
# Embed the training set
embeddings_train = co.embed(texts=sentences_train,
                             model="large",
                             truncate="LEFT").embeddings
# Embed the testing set
embeddings_test = co.embed(texts=sentences_test,
                             model="large",
                             truncate="LEFT").embeddings

In [23]:
print(f"Review text: {sentences_train[0]}")
print(f"Embedding vector: {embeddings_train[0][:10]}")

Review text: if you like quirky , odd movies and or the ironic , here 's a fun one
Embedding vector: [0.8979736, -0.38673842, 1.6181374, 0.3963398, 1.512604, -0.30581218, 2.5686772, -1.9528872, 0.067017235, 1.8820063]


## 3. Train a classifier using the training set

In [24]:
# import SVM classifier code
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler


# Initialize a support vector machine, with class_weight='balanced' because 
# our training set has roughly an equal amount of positive and negative 
# sentiment sentences
svm_classifier = make_pipeline(StandardScaler(), SVC(class_weight='balanced')) 

# fit the support vector machine
svm_classifier.fit(embeddings_train, labels_train)


Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svc',
                 SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
                     decision_function_shape='ovr', degree=3,
                     gamma='auto_deprecated', kernel='rbf', max_iter=-1,
                     probability=False, random_state=None, shrinking=True,
                     tol=0.001, verbose=False))],
         verbose=False)

## 4. Evaluate the performance of the classifier on the testing set

In [26]:
# get the score from the test set, and print it out to screen!
score = svm_classifier.score(embeddings_test, labels_test)
print(f"Validation accuracy on Large is {100*score}%!")

Validation accuracy on Large is 91.2%!
